To begin, let's think of a few things we could do to this model that we'd like to know.  

The most basic things for us to modify are layers and nodes per layer, as well as 0, 1, or 2 dense layers. Let's test those things. 

Lets create a `for` loop to create the planned combinations and print them

In [6]:
import time

dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
#             print(NAME)

Now lets build our model by using the above piece of code along with our model in previous notebook

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("data_model/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("data_model/y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0, 1]
layer_sizes = [32, 64, 128]
conv_layers = [2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=5,
                      validation_split=0.2,
                      callbacks=[tensorboard])
            

2-conv-32-nodes-0-dense-1593961812
Epoch 1/5
623/623 [==============================] - 164s 263ms/step - loss: 0.6075 - accuracy: 0.6554 - val_loss: 0.5935 - val_accuracy: 0.6713
Epoch 2/5
623/623 [==============================] - 163s 261ms/step - loss: 0.5116 - accuracy: 0.7494 - val_loss: 0.5185 - val_accuracy: 0.7488
Epoch 3/5
623/623 [==============================] - 162s 261ms/step - loss: 0.4611 - accuracy: 0.7867 - val_loss: 0.4762 - val_accuracy: 0.7697
Epoch 4/5
623/623 [==============================] - 163s 261ms/step - loss: 0.4271 - accuracy: 0.8053 - val_loss: 0.4665 - val_accuracy: 0.7757
Epoch 5/5
623/623 [==============================] - 162s 260ms/step - loss: 0.3982 - accuracy: 0.8193 - val_loss: 0.4592 - val_accuracy: 0.7825
3-conv-32-nodes-0-dense-1593962636
Epoch 1/5
623/623 [==============================] - 183s 294ms/step - loss: 0.6291 - accuracy: 0.6325 - val_loss: 0.5595 - val_accuracy: 0.7028
Epoch 2/5
623/623 [==============================] - 181s 29

KeyboardInterrupt: 

Obviously, this will take a lot of time depending on your CPU or GPU configuration.

So the idea is to visualize the parameters of each model on Tensorboard, analyse them and select the best set of Convolution layers, Dense layers and Layer sizes that optmizes our model.